# NE Climate Database #
### Plotting Scripts ###

This file collects the plotting scripts from the NE Climate Database file. Plotting will be done in this file, to ease the clutter from the original file. Because IPython notebooks are not "raw" .py files, we first require a handful of methods/classes to import the MWWavelets modules (notebook) that contains all of the wavelet methods.

In [1]:
# Scripts taken from: http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Importing%20Notebooks.html
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

# Importing MWWavelets
Imports the methods required to apply the transforms. Also constructs the NE Climate database.

In [3]:
from MWWavelets import *

importing Jupyter notebook from MWWavelets.ipynb
checkpoint 1
checkpoint 2
Completed Initilization of StateID Object:  62658
Completed Initilization of StateID Object:  63207
Completed Initilization of StateID Object:  67970
Completed Initilization of StateID Object:  68138
Completed Initilization of StateID Object:  170100
Completed Initilization of StateID Object:  170814
Completed Initilization of StateID Object:  171628
Completed Initilization of StateID Object:  172426
Completed Initilization of StateID Object:  172765
Completed Initilization of StateID Object:  173046
Cleaning at ID:  173944
Year: 1901   Value: -9999.0
Set to:  0
Cleaning at ID:  173944
Year: 1901   Value: -9999.0
Set to:  0
Completed Initilization of StateID Object:  173944
Completed Initilization of StateID Object:  174566
Completed Initilization of StateID Object:  175304
Completed Initilization of StateID Object:  176905
Cleaning at ID:  176937
Year: 1900   Value: -9999.0
Set to:  0
Cleaning at ID:  176937
Ye

In [4]:
# this is creating and saving a list containing all of the USHCN IDs so that I can attach lat/long/altitude's to them.
ID_collection = []
for i in StateID_objects:
    ID_collection.append(i.name())

In [5]:
#minimum and maximum longitude for sorting purposes
min_long = 9999
max_long = -9999
for i in StateID_objects:
    current_long = i.coord()[1]
    if min_long > current_long:
        min_long = current_long
    if max_long < current_long:
        max_long = current_long
        
print("Min long is :", min_long)
print("Max long is :", max_long)

Min long is : -80.37
Max long is : -67.0


In [4]:
# Plotting the line to see if it is correct
longitudes = np.linspace(-83.72, -66.51, 1000, endpoint=True)
latitudes = [appalachian_coord(longitudes[i]) for i in range(len(longitudes))]

plt.scatter(longitudes, latitudes)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Appalachian Mountain Range Coordinate Plot")
plt.show()

### Processing and Displaying Data (Plotting Scripts)

In [6]:
#plotting max temp data for 62658
N = 1380
x_val = np.linspace(1900, 2014+11.0/12, N, endpoint=True)
temps = dict_to_list(select_id(62658, StateID_objects).max_temp())

#previously 24, 27/2
my_dpi = 96
plt.figure(num=1, figsize=(11520/my_dpi, 4320/my_dpi), dpi=my_dpi)
plt.subplot(211)
plt.plot(x_val, temps)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 5))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title("Max Temp Data for 62658")

#plotting smoothed max temp data
new_vals = gaussian_smoother(x_val, temps, 2.5)
smoothed_x_vals = new_vals[0]
smoothed_y_vals = new_vals[1]

plt.subplot(212)
plt.plot(smoothed_x_vals, smoothed_y_vals)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 5))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title("Smoothed Max Temp Data for 62658")
plt.show()

### Index and Averaging Methods (Plotting Scripts)

In [4]:
# testing NEI index and average_StateID_series
# StateID_objects is a list of all the StateID objects from the data set
N = 1380
x_val = np.linspace(1900, 2014+11.0/12, N, endpoint=True)
ID62658 = select_id(62658, StateID_objects)
nei_index_62658 = nei_index(ID62658)

plt.figure(num=1, figsize=(24, 27/2), dpi=80)
plt.plot(x_val, nei_index_62658)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 5))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title("NEI Index for 62658")
plt.show()

# averaging for all StateIDs
lists = average_StateID_series(StateID_objects)
avg_max_temps = lists[0]
avg_min_temps = lists[1]
avg_precips = lists[2]
avg_nei = lists[3]

plt.figure(figsize=(24,13))
plt.subplot(2, 2, 1)
plt.plot(x_val, avg_max_temps)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Average Max Temp Data for All StateIDs"
plt.title(title_string_raw)

plt.subplot(2, 2, 2)
plt.plot(x_val, avg_min_temps)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Average Min Temp Data for All StateIDs"
plt.title(title_string_raw)

plt.subplot(2, 2, 3)
plt.plot(x_val, avg_precips)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Average Precip Data for All StateIDs"
plt.title(title_string_raw)

plt.subplot(2, 2, 4)
plt.plot(x_val, avg_nei)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Average NEI Index Data for All StateIDs"
plt.title(title_string_raw)
plt.show()

### Generating Noisy Data (Plotting Scripts)

In [6]:
# Red Noise Generator Example
red_noise_x = np.linspace(0, 100, 100)
red_noise_y = red_noise_generator(0.345084339548, 1, 100)

plt.plot(red_noise_x, red_noise_y,'bx')
plt.xlim(0, 100)
plt.xticks(np.arange(0, 100, 5))
plt.xlabel("Sequence Index")
plt.ylabel("Noise Value")
plt.title("Red Noise for r=0.345084339548, std_dev=1")
plt.show()

In [7]:
# Noisy Line Generator
noisy_x = np.linspace(0, 10, 100)
noise = red_noise_generator(0.345084339548, 1, 100)
line_y = line_generator(-5, 1, noisy_x)
noisy_data = noisy_line_generator(noisy_x, line_y, noise)

plt.plot(noisy_data[0], noisy_data[1])
plt.plot(noisy_x, line_y)
plt.xlim(0, 10)
plt.xticks(np.arange(0, 10, 0.5))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("Linear trend w/ Noisy Data")
plt.show()

# Wavelets

### Sampled Morlet Wavelet Plot

In [5]:
# plotting sampled Morlet wavelet
mlt = Morlet_Wavelet(1, 2*np.pi)
dt = (1.0/12.0)
print(sum([np.absolute(i*i)*dt for i in mlt.morlet_function()]))

x = np.linspace(-2, 2, 2*mlt.sampling_points()+1)
y = np.real(mlt.morlet_function())
z = np.imag(mlt.morlet_function())
print("Wavelet period-year conversion is: "+str(mlt.scale_to_year()))
print("Number of required sampling points: "+str(mlt.sampling_points()))

plt.plot(x, y, 'b', x, z, 'g')
plt.xlim(-3, 3)
plt.xticks(np.arange(-3, 4))
plt.xlabel("x")
plt.ylabel("Amplitude")
plt.title("Morlet Wavelet w/ F = 2*Pi")
save_string = 'sampled_morlet_wavelet'
plt.savefig(save_string)

plt.show()

1.0
Wavelet period-year conversion is: 0.987645853221
Number of required sampling points: 24.0


### Sampled Harr Wavelet Plot

In [6]:
# plotting Harr wavelet
harr = Harr_Wavelet(2)
s = harr.scale()
x = np.linspace(0, 11, 12*s)
y = harr.harr_function()
dt = (1.0/12.0)
print(sum([np.absolute(i*i)*dt for i in mlt.morlet_function()]))
print("Harr wavelet width is (in years): "+str(s))

plt.plot(x, y, 'r')
plt.xlim(0, 11)
plt.xticks(np.arange(0, 12))
plt.xlabel("x")
plt.ylabel("Harr Amplitude")
plt.title("Harr Wavelet Function")

save_string = 'sampled_harr_wavelet'
plt.savefig(save_string)

plt.show()

1.0
Harr wavelet width is (in years): 2


### Convolution Method Testing (Plotting Scripts)

In [10]:
# testing that Harr wavelet convolution method works
station = select_id(62658, StateID_objects)
min_winter_temps_list = dict_to_list(station.max_temp_winter())
N = len(min_winter_temps_list)
x_list = np.linspace(1900, 2014+0.75, N, endpoint=True)
scales = harr_scale_range((1.0/6.0), 10)
transform_coefficients = continuous_transform_harr(min_winter_temps_list, scales)

Z = transform_coefficients

# 24,13 usually
plt.figure(figsize=(6,8))
plt.subplot(2, 1, 1)
plt.plot(x_list, min_winter_temps_list)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 20))
# plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Winter Min Temp Data for " + str(62658)
plt.title(title_string_raw)

plt.subplot(2, 1, 2)
plt.contourf(x_list, scales, Z)
plt.xlabel("Year")
plt.xticks(np.arange(1900, 2016, 20))
plt.ylabel("Scale")
plt.title("Harr Wavelet Coefficients, Raw Data")
# cbar = plt.colorbar()
save_string = 'harr_wavelet_test_thesis.jpg'
plt.savefig(save_string)

plt.show()

# Plotting Wavelet Transform Coefficients

In [5]:
ID = 272999
N = 1380
kernel_std = 1.25
kernel_width = 4*kernel_std
temps = dict_to_list(select_id(ID, StateID_objects).min_temp())
x_list = np.linspace(1900, 2014+11.0/12, N, endpoint=True)
smoothed_temps = gaussian_smoother(x_list, temps, kernel_std)

"""
# testing on small scale, medium scale, and large scale wavelets
mlt_small = Morlet_Wavelet(1, 2*np.pi)
mlt_med = Morlet_Wavelet(10, 2*np.pi)
mlt_large = Morlet_Wavelet(25, 2*np.pi)

small_coefficients = continuous_convolution(temps, mlt_small.morlet_function())
med_coefficients = continuous_convolution(temps, mlt_med.morlet_function())
large_coefficients = continuous_convolution(temps, mlt_large.morlet_function())
print('-----small sampled values-----')
print(len(mlt_small.morlet_function()))
print(mlt_small.scale_to_year())
print(mlt_small.sampling_points())
print('-----med sampled values-----')
print(len(mlt_med.morlet_function()))
print(mlt_med.scale_to_year())
print(mlt_med.sampling_points())
print('------large sampled values-----')
print(len(mlt_large.morlet_function()))
print(mlt_large.scale_to_year())
print(mlt_large.sampling_points())

small_power = wavelet_power_spectra([small_coefficients])[0]
med_power = wavelet_power_spectra([med_coefficients])[0]
large_power = wavelet_power_spectra([large_coefficients])[0]

N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
plt.plot(X, small_power, '-r', X, med_power, '-g', X, large_power, '-b')
plt.xlabel("Year")
plt.ylabel("Wavelet Power")
plt.title("Wavelet Power for three scaled wavelets")
plt.show()
"""

transform_coefficients = continuous_transform_morlet(temps, scale_range(0.5, 30), 2*np.pi)
transform_power = wavelet_power_spectra(transform_coefficients)
print('length of transform coefficients is: '+str(len(transform_coefficients)))
print('number of coefficients at lowest scale: '+str(len(transform_coefficients[0])))

# smoothing data before transforming
# NOTE: This is not a good idea, smooth wavelet coefficients after transforming instead!
transform_coefficients_smooth = continuous_transform_morlet(smoothed_temps[1], scale_range(0.5, 30), 2*np.pi)
transform_power_smooth = wavelet_power_spectra(transform_coefficients_smooth)
# takes approximately 2.5 minutes per transform

# renormalizing wavelet power with sample variance
variance = sample_variance(temps)
normalized_powers = normalized_wavelet_power_spectra(transform_power, variance)

length of transform coefficients is: 119
number of coefficients at lowest scale: 1380


In [12]:
print(len(transform_power))
print(len(normalized_powers))

print(len(transform_power[0]))
print(len(normalized_powers[0]))

print(variance)

119
119
1380
1380
1.0470612712424183


In [14]:
# plotting wavelet power at lowest and highest scale w.r.t time
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
Y = transform_power[-1]
Z = transform_power[0]
plt.plot(X, Y, '-r', X, Z, '-g')
plt.xlabel("Year")
plt.ylabel("Coefficient Power")
plt.title("Wavelet Power at lowest scale")
plt.legend(['Highest scale', 'Lowest scale'])
plt.show()

In [9]:
# plotting coefficients
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
Y = scale_range(0.5, 30)
Z = transform_power
Z2 = transform_power_smooth

# full screen is 24,13
plt.figure(figsize=(24,13))
plt.subplot(2, 2, 1)
plt.plot(X, temps)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 5))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Annual Min Temp Data for " + str(ID)
plt.title(title_string_raw)

plt.subplot(2, 2, 3)
plt.plot(smoothed_temps[0], smoothed_temps[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 5))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_gauss_smooth = "Smoothed w/ Gaussian kernel, width "+ str(kernel_width) + " years"
plt.title(title_string_gauss_smooth)

plt.subplot(2, 2, 2)
plt.contourf(X, Y, Z)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Morlet Wavelet Coefficients, Raw Data")
cbar = plt.colorbar()

plt.subplot(2, 2, 4)
plt.contourf(smoothed_temps[0], Y, Z2)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Morlet Wavelet Coefficients, Smoothed Data")
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'Morlet_Annual_Min_Temp_'+str(ID)+'_Gauss_width_'+str(int(np.floor(kernel_width)))
#plt.savefig(save_string)
plt.show()

In [65]:
# FOR THESIS, NEED SMALLER PLOTS TO PUT ON PAGE. ONLY DOING TIME SERIES ABOVE WAVELET PLOT FOR ONE METRIC AT A TIME.
# plotting coefficients
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
Y = scale_range(0.5, 30)
Z = transform_power
Z2 = transform_power_smooth

# plots for thesis have 6 inch width
plt.figure(figsize=(6, 8))
plt.subplot(2, 1, 1)
plt.plot(X, temps)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 20))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Annual Min Temp Data for " + str(ID)
plt.title(title_string_raw)

plt.subplot(2, 1, 2)
plt.contourf(X, Y, Z)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Wavelet Coefficients, Raw Data")

plt.tight_layout()
save_string = 'Morlet_Annual_Min_Temp_'+str(ID)+'_Gauss_width_'+str(int(np.floor(kernel_width)))+'_thesis'
plt.savefig(save_string)

# smoothing time series
plt.figure(figsize=(6,8))
plt.subplot(2, 1, 1)
plt.plot(smoothed_temps[0], smoothed_temps[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 20))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_gauss_smooth = "S("+ str(kernel_width) + "yr), Min Temp"
plt.title(title_string_gauss_smooth)

plt.subplot(2, 1, 2)
plt.contourf(smoothed_temps[0], Y, Z2)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Coefficients, Presmoothed Data")

plt.tight_layout()
save_string = 'Morlet_Annual_Min_Temp_'+str(ID)+'_Gauss_width_'+str(int(np.floor(kernel_width)))+'_presmoothed_thesis'
plt.savefig(save_string)

# smoothing coefficients rather than time series
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
Y = scale_range(0.5, 30)
Z = transform_power
smoothed_wavelet_power = wavelet_power_smoothing(X, Z, kernel_std)
Z2 = smoothed_wavelet_power[1]

plt.figure(figsize=(6, 8))
plt.subplot(2, 1, 1)
plt.plot(smoothed_temps[0], smoothed_temps[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 20))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_gauss_smooth = "S("+ str(kernel_width) + "yr), Min Temp"
plt.title(title_string_gauss_smooth)

plt.subplot(2, 1, 2)
plt.contourf(smoothed_temps[0], Y, Z2)
plt.xlabel("Year")
plt.ylabel("Scale")
title_string_gauss_smooth = "S(5) Coefficients"
plt.title(title_string_gauss_smooth)

plt.tight_layout()
save_string = 'Morlet_Annual_Min_Temp_Smoothed_Wavelets_'+str(ID)+'_Gauss_width_'+str(int(np.floor(kernel_width)))+'_thesis'
plt.savefig(save_string)

In [57]:
# smoothing coefficients rather than time series
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
Y = scale_range(0.5, 30)
Z = transform_power
smoothed_wavelet_power = wavelet_power_smoothing(X, Z, kernel_std)
Z2 = smoothed_wavelet_power[1]

plt.figure(figsize=(24,13))
plt.subplot(2, 2, 1)
plt.plot(X, temps)
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Annual Min Temp Data for " + str(ID)
plt.title(title_string_raw)

plt.subplot(2, 2, 3)
plt.plot(smoothed_temps[0], smoothed_temps[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_gauss_smooth = "Smoothed w/ Gaussian kernel, width "+ str(kernel_width) + " years"
plt.title(title_string_gauss_smooth)

plt.subplot(2, 2, 2)
plt.contourf(X, Y, Z)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Morlet Wavelet Coefficients, Raw Data")
cbar = plt.colorbar()

plt.subplot(2, 2, 4)
plt.contourf(smoothed_temps[0], Y, Z2)
plt.xlabel("Year")
plt.ylabel("Scale")
title_string_gauss_smooth = "Smoothed w/ Gaussian kernel, width "+ str(kernel_width) + " years"
plt.title(title_string_gauss_smooth)
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'Morlet_Annual_Min_Temp_Smoothed_Wavelets_'+str(ID)+'_Gauss_width_'+str(int(np.floor(kernel_width)))
#plt.savefig(save_string)
plt.show()

In [19]:
# adding COI to above code to test
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
Y = scale_range(0.5, 30)
Y = scale_range_years(Y, 2*np.pi)
Z = transform_power
smoothed_wavelet_power = wavelet_power_smoothing(X, Z, kernel_std)
Z2 = smoothed_wavelet_power[1]
coi_data_left = coi_left(X[0], Y)
coi_data_right = coi_right(X[-1], Y)

fig = plt.figure(figsize=(24,13))
ax1 = plt.subplot(2, 2, 1)
ax1.plot(X, temps)
ax1.set_xlim(1895, 2020)
ax1.set_xticks(np.arange(1900, 2016, 5))
ax1.set_xlabel("Year")
ax1.set_ylabel("Z-score")
title_string_raw = "Annual Min Temp Data for " + str(ID)
ax1.set_title(title_string_raw)

ax3 = plt.subplot(2, 2, 3)
ax3.plot(smoothed_temps[0], smoothed_temps[1])
ax3.set_xlim(1895, 2020)
ax3.set_xticks(np.arange(1900, 2016, 5))
ax3.set_xlabel("Year")
ax3.set_ylabel("Z-score")
title_string_gauss_smooth = "Smoothed w/ Gaussian kernel, width "+ str(kernel_width) + " years"
ax3.set_title(title_string_gauss_smooth)

ax2 = plt.subplot(2, 2, 2)
ax2.contourf(X, Y, Z)
ax2.add_patch(Polygon(coi_data_left, closed=True,fill=False,hatch='x'))
ax2.add_patch(Polygon(coi_data_right, closed=True,fill=False,hatch='x'))
ax2.set_xlabel("Year")
ax2.set_ylabel("Wavelet Period [Years]")
ax2.set_title("Morlet Wavelet Coefficients, Raw Data")

ax4 = plt.subplot(2, 2, 4)
ax4.contourf(smoothed_wavelet_power[0], Y, Z2)
ax4.add_patch(Polygon(coi_data_left, closed=True,fill=False,hatch='\\'))
ax4.add_patch(Polygon(coi_data_right, closed=True,fill=False,hatch='/'))
ax4.set_xlabel("Year")
ax4.set_ylabel("Wavelet Period [Years]")
title_string_gauss_smooth = "Smoothed w/ Gaussian kernel, width "+ str(kernel_width) + " years"
ax4.set_title(title_string_gauss_smooth)

fig.tight_layout()
save_string = 'Morlet_Annual_Min_Temp_Smoothed_Wavelets_'+str(ID)+'_Gauss_width_'+str(int(np.floor(kernel_width)))
#plt.savefig(save_string)
plt.show()

In [8]:
#DELETE THIS AFTER MAKING PLOT
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
Y = scale_range(0.5, 30)
Y = scale_range_years(Y, 2*np.pi)
Z = transform_power
coi_data_left = coi_left(X[0], Y)
coi_data_right = coi_right(X[-1], Y)

fig = plt.figure(figsize=(10, 5.625))

ax2 = plt.subplot(1, 1, 1)
ax2.contourf(X, Y, Z)
ax2.add_patch(Polygon(coi_data_left, closed=True,fill=False,hatch='x'))
ax2.add_patch(Polygon(coi_data_right, closed=True,fill=False,hatch='x'))
ax2.set_xlabel("Year")
ax2.set_ylabel("Wavelet Period [Years]")
ax2.set_title("Morlet Wavelet Coefficients, Min Temp ")

fig.tight_layout()
save_string = 'Morlet_Annual_Min_Temp_Smoothed_Wavelets_'+str(ID)+'_Gauss_width_'+str(int(np.floor(kernel_width)))+'_thesis_lowres'
plt.savefig(save_string, dpi = 2*192)
plt.show()

In [20]:
# NOTE: THIS CODE IS NOT APPLICABLE, USES INAPPLICABLE STATISTICAL TESTING METHODS. SEE MWWAVELETS NOTES ON STAT TESTING.

# testing baseline subtraction code
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
Y = scale_range(0.5, 30)
Y = scale_range_years(Y, 2*np.pi)
Z = normalized_powers
smoothed_wavelet_power = wavelet_power_smoothing(X, Z, kernel_std)
alpha = (autocorrelation(1, temps) + np.sqrt(autocorrelation(2, temps)))/2
Z2 = base_power_subtraction(Z, Y, alpha)
coi_data_left = coi_left(X[0], Y)
coi_data_right = coi_right(X[-1], Y)

plt.figure(figsize=(24,13))

plt.subplot(2, 1, 1)
plt.contourf(X, Y, Z)
plt.xlabel("Year")
plt.ylabel("Wavelet Period [Years]")
plt.title("Morlet Wavelet Coefficients, Raw Data")
cbar = plt.colorbar()

plt.subplot(2, 1, 2)
plt.contourf(X, Y, Z2)
plt.xlabel("Year")
plt.ylabel("Wavelet Period [Years]")
plt.title("Morlet Wavelet Coefficients, Subtracted Data")
plt.colorbar()

fig.tight_layout()
save_string = 'Morlet_Annual_Min_Temp_Smoothed_Wavelets_'+str(ID)+'_Gauss_width_'+str(int(np.floor(kernel_width)))
#plt.savefig(save_string)
plt.show()

In [21]:
# this was to test making a filled shape in matplotlib
plt.figure()
plt.fill([1, 3, 2, 1], [1, 1, 2, 2], fill=False, hatch='\\')
plt.show()

### Wavelet transform on Appalachian Sorted Data ###

In [6]:
# Plotting positions of StateID stations
west_stations = Appalachian_StateID_objects[0]
x_west_data = [i.coord()[1] for i in west_stations]
y_west_data = [i.coord()[0] for i in west_stations]

east_stations = Appalachian_StateID_objects[1]
x_east_data = [i.coord()[1] for i in east_stations]
y_east_data = [i.coord()[0] for i in east_stations]

plt.scatter(x_west_data, y_west_data, c='red')
plt.scatter(x_east_data, y_east_data, c='blue')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Distribution of West vs East StateIDs')
#save_string = 'Distribution of West vs East StateIDs.jpg'
#plt.savefig(save_string)
plt.show()

# compiling data for plotting all parameters
# west data
west_data = average_StateID_series(west_stations)
west_maxtemp_data = west_data[0]
west_mintemp_data = west_data[1]
west_precip_data = west_data[2]
west_nei_data = west_data[3]

# east data
east_data = average_StateID_series(east_stations)
east_maxtemp_data = east_data[0]
east_mintemp_data = east_data[1]
east_precip_data = east_data[2]
east_nei_data = east_data[3]

In [7]:
kernel_std = 1.25
kernel_width = 4*kernel_std
scales = scale_range(0.5, 25)
nei_years = scale_range_years(scales, 2*np.pi)
x_list = np.linspace(1900, 2014+11.0/12, 1380, endpoint=True)

# For West Appalachian Stations
# max_temp
start_time = time.perf_counter()
west_maxtemp_transform_coeff = continuous_transform_morlet(west_maxtemp_data, scales, 2*np.pi)
west_maxtemp_transform_power = wavelet_power_spectra(west_maxtemp_transform_coeff)

west_maxtemp_smoothed_wavelet_power = wavelet_power_smoothing(x_list, west_maxtemp_transform_power, kernel_std)
west_maxtemp_smoothed_wavelet_power_x = west_maxtemp_smoothed_wavelet_power[0]
west_maxtemp_smoothed_wavelet_power_coeffs = west_maxtemp_smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

# min_temp
start_time = time.perf_counter()
west_mintemp_transform_coeff = continuous_transform_morlet(west_mintemp_data, scales, 2*np.pi)
west_mintemp_transform_power = wavelet_power_spectra(west_mintemp_transform_coeff)

west_mintemp_smoothed_wavelet_power = wavelet_power_smoothing(x_list, west_mintemp_transform_power, kernel_std)
west_mintemp_smoothed_wavelet_power_x = west_mintemp_smoothed_wavelet_power[0]
west_mintemp_smoothed_wavelet_power_coeffs = west_mintemp_smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

#precip
start_time = time.perf_counter()
west_precip_transform_coeff = continuous_transform_morlet(west_precip_data, scales, 2*np.pi)
west_precip_transform_power = wavelet_power_spectra(west_precip_transform_coeff)

west_precip_smoothed_wavelet_power = wavelet_power_smoothing(x_list, west_precip_transform_power, kernel_std)
west_precip_smoothed_wavelet_power_x = west_precip_smoothed_wavelet_power[0]
west_precip_smoothed_wavelet_power_coeffs = west_precip_smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

#nei
start_time = time.perf_counter()
west_nei_transform_coeff = continuous_transform_morlet(west_nei_data, scales, 2*np.pi)
west_nei_transform_power = wavelet_power_spectra(west_nei_transform_coeff)

west_nei_smoothed_wavelet_power = wavelet_power_smoothing(x_list, west_nei_transform_power, kernel_std)
west_nei_smoothed_wavelet_power_x = west_nei_smoothed_wavelet_power[0]
west_nei_smoothed_wavelet_power_coeffs = west_nei_smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

# For East Appalacian Stations
# max_temp
start_time = time.perf_counter()
east_maxtemp_transform_coeff = continuous_transform_morlet(east_maxtemp_data, scales, 2*np.pi)
east_maxtemp_transform_power = wavelet_power_spectra(east_maxtemp_transform_coeff)

east_maxtemp_smoothed_wavelet_power = wavelet_power_smoothing(x_list, east_maxtemp_transform_power, kernel_std)
east_maxtemp_smoothed_wavelet_power_x = east_maxtemp_smoothed_wavelet_power[0]
east_maxtemp_smoothed_wavelet_power_coeffs = east_maxtemp_smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

# min_temp
start_time = time.perf_counter()
east_mintemp_transform_coeff = continuous_transform_morlet(east_mintemp_data, scales, 2*np.pi)
east_mintemp_transform_power = wavelet_power_spectra(east_mintemp_transform_coeff)

east_mintemp_smoothed_wavelet_power = wavelet_power_smoothing(x_list, east_mintemp_transform_power, kernel_std)
east_mintemp_smoothed_wavelet_power_x = east_mintemp_smoothed_wavelet_power[0]
east_mintemp_smoothed_wavelet_power_coeffs = east_mintemp_smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

# precip
start_time = time.perf_counter()
east_precip_transform_coeff = continuous_transform_morlet(east_precip_data, scales, 2*np.pi)
east_precip_transform_power = wavelet_power_spectra(east_precip_transform_coeff)

east_precip_smoothed_wavelet_power = wavelet_power_smoothing(x_list, east_precip_transform_power, kernel_std)
east_precip_smoothed_wavelet_power_x = east_precip_smoothed_wavelet_power[0]
east_precip_smoothed_wavelet_power_coeffs = east_precip_smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

# nei
start_time = time.perf_counter()
east_nei_transform_coeff = continuous_transform_morlet(east_nei_data, scales, 2*np.pi)
east_nei_transform_power = wavelet_power_spectra(east_nei_transform_coeff)

east_nei_smoothed_wavelet_power = wavelet_power_smoothing(x_list, east_nei_transform_power, kernel_std)
east_nei_smoothed_wavelet_power_x = east_nei_smoothed_wavelet_power[0]
east_nei_smoothed_wavelet_power_coeffs = east_nei_smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

Time Elapsed: 114.32743085045877
Time Elapsed: 113.96092951489479
Time Elapsed: 116.67743328808751
Time Elapsed: 114.34703879422051
Time Elapsed: 114.46593563462602
Time Elapsed: 112.39104911266031
Time Elapsed: 113.78998502355091
Time Elapsed: 134.91572732427903


In [13]:
# Plotting time series from above
N = 1380
X = np.linspace(1900, 2014+11/12, N, endpoint=True)
my_dpi = float(96)

kernel_std = 1.25
# maxtemp_data
west_maxtemp_data_smooth = gaussian_smoother(X, west_maxtemp_data, kernel_std)
east_maxtemp_data_smooth = gaussian_smoother(X, east_maxtemp_data, kernel_std)
fig = plt.figure(figsize=(50, 22.5))
plt.subplot(2,1,1)
plt.plot(west_maxtemp_data_smooth[0], west_maxtemp_data_smooth[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title('Averaged Monthly Maximum Temperature, West of Appalacians, Smoothed w/ 5 Year Gaussian')

plt.subplot(2,1,2)
plt.plot(east_maxtemp_data_smooth[0], east_maxtemp_data_smooth[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title('Averaged Monthly Maximum Temperature, East of Appalacians, Smoothed w/ 5 Year Gaussian')

plt.tight_layout()

save_string = 'West vs East of Appalachian Mountain Max Temp_presentation_timeseries.jpg'
plt.savefig(save_string, dpi=my_dpi)
plt.gcf().clear()

west_mintemp_data_smooth = gaussian_smoother(X, west_mintemp_data, kernel_std)
east_mintemp_data_smooth = gaussian_smoother(X, east_mintemp_data, kernel_std)
fig = plt.figure(figsize=(25, 11.25))
plt.subplot(2,1,1)
plt.plot(west_mintemp_data_smooth[0], west_mintemp_data_smooth[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title('Averaged Monthly Minimum Temperature, West of Appalacians, Smoothed w/ 5 Year Gaussian')

plt.subplot(2,1,2)
plt.plot(east_mintemp_data_smooth[0], east_mintemp_data_smooth[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title('Averaged Monthly Minimum Temperature, East of Appalacians, Smoothed w/ 5 Year Gaussian')

plt.tight_layout()

save_string = 'West vs East of Appalachian Mountain Min Temp_presentation_timeseries.jpg'
plt.savefig(save_string, dpi=2*my_dpi)
plt.gcf().clear()

west_precip_data_smooth = gaussian_smoother(X, west_precip_data, kernel_std)
east_precip_data_smooth = gaussian_smoother(X, east_precip_data, kernel_std)
fig = plt.figure(figsize=(25, 11.25))
plt.subplot(2,1,1)
plt.plot(west_precip_data_smooth[0], west_precip_data_smooth[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title('Averaged Monthly Precipitation, West of Appalacians, Smoothed w/ 5 Year Gaussian')

plt.subplot(2,1,2)
plt.plot(east_precip_data_smooth[0], east_precip_data_smooth[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title('Averaged Monthly Precipitation, East of Appalacians, Smoothed w/ 5 Year Gaussian')

plt.tight_layout()

save_string = 'West vs East of Appalachian Mountain Precip_presentation_timeseries.jpg'
plt.savefig(save_string, dpi=2*my_dpi)
plt.gcf().clear()

west_nei_data_smooth = gaussian_smoother(X, west_nei_data, kernel_std)
east_nei_data_smooth = gaussian_smoother(X, east_nei_data, kernel_std)
fig = plt.figure(figsize=(25, 11.25))
plt.subplot(2,1,1)
plt.plot(west_nei_data_smooth[0], west_nei_data_smooth[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title('Averaged Monthly NEI, West of Appalacians, Smoothed w/ 5 Year Gaussian')

plt.subplot(2,1,2)
plt.plot(east_nei_data_smooth[0], east_nei_data_smooth[1])
plt.xlim(1895, 2020)
plt.xticks(np.arange(1900, 2016, 10))
plt.xlabel("Year")
plt.ylabel("Z-score")
plt.title('Averaged Monthly NEI, East of Appalacians, Smoothed w/ 5 Year Gaussian')

plt.tight_layout()

save_string = 'West vs East of Appalachian Mountain NEI_presentation_timeseries.jpg'
plt.savefig(save_string, dpi=2*my_dpi)
plt.gcf().clear()

plt.show()

In [14]:
# Plotting Coefficients from above
# doing pairs of plots, one for each parameter, comparing between West/East stations (gives 4 plots)
my_dpi = float(96)
# maxtemp
plt.gcf().clear()
#FOR THESIS, 6,8, for full screen, 24,13
#FOR PRESENTATION, 5760/dpi, 2160/dpi (my_dpi=96) (50 by 22.5in)
# also for presentation, want a 
fig = plt.figure(figsize=(25, 11.25))
plt.subplot(2,1,1)
plt.contourf(west_maxtemp_smoothed_wavelet_power_x, nei_years, west_maxtemp_smoothed_wavelet_power_coeffs)
plt.xlabel("Year")
plt.ylabel("Scale [Years]")
plt.title("Morlet Wavelet Coefficients for Max Temp, West of Appalachians")
#plt.title("Max Temp, West") # for thesis
cbar = plt.colorbar()

plt.subplot(2,1,2)
plt.contourf(east_maxtemp_smoothed_wavelet_power_x, nei_years, east_maxtemp_smoothed_wavelet_power_coeffs)
plt.xlabel("Year")
plt.ylabel("Scale [Years]")
plt.title("Morlet Wavelet Coefficients for Max Temp, East of Appalachians")
#plt.title("Max Temp, East") # for thesis
cbar = plt.colorbar()
fig.tight_layout()

save_string = 'West vs East of Appalachian Mountain Max Temp_presentation.jpg'
plt.savefig(save_string, dpi=2*my_dpi)
plt.gcf().clear()

# mintemp
fig = plt.figure(figsize=(25, 11.25))
plt.subplot(2,1,1)
plt.contourf(west_mintemp_smoothed_wavelet_power_x, nei_years, west_mintemp_smoothed_wavelet_power_coeffs)
plt.xlabel("Year")
plt.ylabel("Scale [Years]")
plt.title("Morlet Wavelet Coefficients for Min Temp, West of Appalacians")
#plt.title("Min Temp, West") # thesis
cbar = plt.colorbar()

plt.subplot(2,1,2)
plt.contourf(east_mintemp_smoothed_wavelet_power_x, nei_years, east_mintemp_smoothed_wavelet_power_coeffs)
plt.xlabel("Year")
plt.ylabel("Scale [Years]")
plt.title("Morlet Wavelet Coefficients for Min Temp, East of Appalachians")
#plt.title("Min Temp, East") # thesis
cbar = plt.colorbar()

fig.tight_layout()

save_string = 'West vs East of Appalachian Mountain Min Temp_presentation.jpg'
plt.savefig(save_string, dpi=2*my_dpi)
plt.gcf().clear()

# precip
fig = plt.figure(figsize=(25, 11.25))
plt.subplot(2,1,1)
plt.contourf(west_precip_smoothed_wavelet_power_x, nei_years, west_precip_smoothed_wavelet_power_coeffs)
plt.xlabel("Year")
plt.ylabel("Scale [Years]")
plt.title("Morlet Wavelet Coefficients for Precip, West of Appalachians")
#plt.title("Precip, West")
cbar = plt.colorbar()

plt.subplot(2,1,2)
plt.contourf(east_precip_smoothed_wavelet_power_x, nei_years, east_precip_smoothed_wavelet_power_coeffs)
plt.xlabel("Year")
plt.ylabel("Scale [Years]")
plt.title("Morlet Wavelet Coefficients for Precip, East of Appalachians")
#plt.title("Precip, East")
cbar = plt.colorbar()

fig.tight_layout()

save_string = 'West vs East of Appalachian Mountain Precip_presentation.jpg'
plt.savefig(save_string, dpi=2*my_dpi)
plt.gcf().clear()

# nei
fig = plt.figure(figsize=(25, 11.25))
plt.subplot(2,1,1)
plt.contourf(west_nei_smoothed_wavelet_power_x, nei_years, west_nei_smoothed_wavelet_power_coeffs)
plt.xlabel("Year")
plt.ylabel("Scale [Years]")
plt.title("Morlet Wavelet Coefficients for NEI, West of Appalachians")
#plt.title("NEI, West")
cbar = plt.colorbar()

plt.subplot(2,1,2)
plt.contourf(east_nei_smoothed_wavelet_power_x, nei_years, east_nei_smoothed_wavelet_power_coeffs)
plt.xlabel("Year")
plt.ylabel("Scale [Years]")
plt.title("Morlet Wavelet Coefficients for NEI, East of Appalachians")
#plt.title("NEI, East")
cbar = plt.colorbar()

fig.tight_layout()

save_string = 'West vs East of Appalachian Mountain NEI_presentation.jpg'
plt.savefig(save_string, dpi=2*my_dpi)
plt.gcf().clear()
plt.show()
#could do a 4-part-plot where stations w/ coordinates are plotted on the right, corresponding wavelets on left
# also a 2 row by 3 column huge plot w/ coordinates on far right, averaged indices in middle, wavelets on the left

C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


### Testing Wavelet Transform on Noisy Data

In [48]:
x_list = np.linspace(0, 100, 1000)
y_list_noise = red_noise_generator(0.345084339548, 2, 1000)
y_list_values = line_generator(-3, 0.06, x_list)
noisy_data = noisy_line_generator(x_list, y_list_values, y_list_noise)

noisy_scales = scale_range(0.5, 30)
noisy_coefficients = continuous_transform_morlet(noisy_data[1], noisy_scales, 2*np.pi)
noisy_transform_power = wavelet_power_spectra(noisy_coefficients)

noisy_flat_coefficients = continuous_transform_morlet(y_list_noise, noisy_scales, 2*np.pi)
noisy_flat_transform_power = wavelet_power_spectra(noisy_flat_coefficients)
# takes about 6.5 minutes

In [23]:
plt.figure(figsize=(24,13))
plt.subplot(2,2,1)
plt.plot(noisy_data[0], noisy_data[1])
plt.xlim(0, 100)
plt.xticks(np.arange(0, 100, 5))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("Linear trend w/ Noisy Data")

plt.subplot(2,2,3)
plt.plot(noisy_data[0], y_list_noise)
plt.xlim(0, 100)
plt.xticks(np.arange(0, 100, 5))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("No trend w/ Noisy Data")

plt.subplot(2,2,2)
plt.contourf(noisy_data[0], noisy_scales, noisy_transform_power)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Morlet Wavelet Coefficients, Noisy Data")
cbar = plt.colorbar()

plt.subplot(2,2,4)
plt.contourf(noisy_data[0], noisy_scales, noisy_flat_transform_power)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Morlet Wavelet Coefficients, Noisy Data w/o Trend")
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'Noisy Data r = 0.345084339548 Test.jpg'
#plt.savefig(save_string)
plt.show()

In [49]:
# FOR THESIS
plt.figure(figsize=(6,8))
plt.subplot(2,1,1)
plt.plot(noisy_data[0], noisy_data[1])
plt.xlim(0, 100)
plt.xticks(np.arange(0, 100, 5))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("Linear trend Noisy Data")

plt.subplot(2,1,2)
plt.contourf(noisy_data[0], noisy_scales, noisy_transform_power)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Wavelet Coefficients, Noisy")
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'Noisy Data r = 0.345084339548 Test_trend_thesis.jpg'
plt.savefig(save_string)

plt.figure(figsize=(6,8))
plt.subplot(2,1,1)
plt.plot(noisy_data[0], y_list_noise)
plt.xlim(0, 100)
plt.xticks(np.arange(0, 100, 5))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("No trend Noisy Data")

plt.subplot(2,1,2)
plt.contourf(noisy_data[0], noisy_scales, noisy_flat_transform_power)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Wavelet Coefficients, Noisy w/o Trend")
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'Noisy Data r = 0.345084339548 Test_no_trend_thesis.jpg'
plt.savefig(save_string)

### Wavelet Transforms on Sines w/ Trends

In [50]:
# transforming the data with timing
time.clock()
length = 100
x_list = np.linspace(0, length, 10*length)
freq = 2*np.pi/10
slope = 0.03
y_list_sine = list(np.sin(freq*x_list))
y_list_trend = line_generator(-1.5, slope, x_list)
y_list_sinetrend = []
for i in range(len(y_list_sine)):
    y_list_sinetrend.append(y_list_sine[i]+y_list_trend[i])

sine_scales = scale_range(0.5, 50)
sine_years = scale_range_years(sine_scales, 2*np.pi)
sine_coefficients = continuous_transform_morlet(y_list_sine, sine_scales, 2*np.pi)
sine_transform_power = wavelet_power_spectra(sine_coefficients)

sinetrend_coefficients = continuous_transform_morlet(y_list_sinetrend, sine_scales, 2*np.pi)
sinetrend_transform_power = wavelet_power_spectra(sinetrend_coefficients)
time.clock()

3462.080049026702

In [25]:
# plotting the coefficients
plt.figure(figsize=(24,13))
plt.subplot(2,2,1)
plt.plot(x_list, y_list_sine)
plt.xlim(0, length)
plt.xticks(np.arange(0, length, 5))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("Sine w/o Trend, frequency: "+str(freq))

plt.subplot(2,2,3)
plt.plot(x_list, y_list_sinetrend)
plt.xlim(0, length)
plt.xticks(np.arange(0, length, 5))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("Sine w/ Trend, slope: "+str(slope))

plt.subplot(2,2,2)
plt.contourf(x_list, sine_years, sine_transform_power)
plt.xticks(np.arange(0, length, 5))
plt.xlabel("Index")
plt.ylabel("Period [Years]")
plt.title("Morlet Wavelet Coefficients, Sine")
cbar = plt.colorbar()

plt.subplot(2,2,4)
plt.contourf(x_list, sine_years, sinetrend_transform_power)
plt.xticks(np.arange(0, length, 5))
plt.xlabel("Index")
plt.ylabel("Period [Years]")
plt.title("Morlet Wavelet Coefficients, Sine w/ Trend")
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'Sine '+str(freq)+' slope '+str(slope)+' length '+str(length)+'.jpg'
#plt.savefig(save_string)
plt.show()

In [52]:
# FOR THESIS
plt.figure(figsize=(6,8))
plt.subplot(2,1,1)
plt.plot(x_list, y_list_sine)
plt.xlim(0, length)
plt.xticks(np.arange(0, length, 10))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("Sine w/o Trend, freq: "+str(freq))

plt.subplot(2,1,2)
plt.contourf(x_list, sine_years, sine_transform_power)
plt.xticks(np.arange(0, length, 10))
plt.xlabel("Index")
plt.ylabel("Period [Years]")
plt.title("Wavelet Coefficients, Sine")
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'Sine '+str(freq)+' slope '+str(slope)+' length '+str(length)+'_no_trend_thesis.jpg'
plt.savefig(save_string)

plt.figure(figsize=(6,8))
plt.subplot(2,1,1)
plt.plot(x_list, y_list_sinetrend)
plt.xlim(0, length)
plt.xticks(np.arange(0, length, 10))
plt.xlabel("Sequence Index")
plt.ylabel("Y Value")
plt.title("Sine w/ Trend, slope: "+str(slope))

plt.subplot(2,1,2)
plt.contourf(x_list, sine_years, sinetrend_transform_power)
plt.xticks(np.arange(0, length, 10))
plt.xlabel("Index")
plt.ylabel("Period [Years]")
plt.title("Wavelet Coefficients, Sine w/ Trend")
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'Sine '+str(freq)+' slope '+str(slope)+' length '+str(length)+'_trend_thesis.jpg'
plt.savefig(save_string)

# Wavelet Transforms on NEI Averaged Data

In [46]:
# names of data sets for all Northeast averaged data:
# avg_nei, avg_max_temps, avg_min_temps, avg_precips
start_time = time.perf_counter()
data = avg_nei
kernel_std = 1.25
kernel_width = 4*kernel_std
scales = scale_range(0.5, 25)
nei_years = scale_range_years(scales, 2*np.pi)
x_list = np.linspace(1900, 2014+11.0/12, 1380, endpoint=True)

transform_coeff = continuous_transform_morlet(data, scales, 2*np.pi)
transform_power = wavelet_power_spectra(transform_coeff)

smoothed_wavelet_power = wavelet_power_smoothing(x_list, transform_power, kernel_std)
smoothed_wavelet_power_x = smoothed_wavelet_power[0]
smoothed_wavelet_power_coeffs = smoothed_wavelet_power[1]
end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

Time Elapsed: 76.19479046463948


In [40]:
# plotting smoothed data
smoothed_data = gaussian_smoother(x_list, data, kernel_std)

fig = plt.figure(figsize=(18,5))
plt.plot(smoothed_data[0], smoothed_data[1])
plt.xlim(1900, 2016)
plt.xticks(np.arange(1900, 2016, 5))
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_gauss_smooth = "Smoothed w/ Gaussian kernel, width "+ str(kernel_width) + " years"
plt.title(title_string_gauss_smooth)
save_string = "max_temp_data_smoothed_all_stations"
#plt.savefig(save_string)
plt.show()

In [41]:
# plotting transform
coi_data_left = coi_left(x_list[0], nei_years)
coi_data_right = coi_right(x_list[-1], nei_years)

#size is usually 24,13
fig = plt.figure(figsize=(18,10))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(x_list, data)
ax1.set_xlim(1900, 2016)
ax1.set_xticks(np.arange(1900, 2016, 5))
ax1.set_xlabel("Year")
ax1.set_ylabel("Z-score")
title_string_raw = "Average Max Temp Data for All StateIDs"
ax1.set_title(title_string_raw)

ax2 = plt.subplot(2, 1, 2)
ax2.contourf(x_list, nei_years, transform_power)
ax2.add_patch(Polygon(coi_data_left, closed=True,fill=False,hatch='\\'))
ax2.add_patch(Polygon(coi_data_right, closed=True,fill=False,hatch='/'))
ax2.set_xlabel("Year")
ax2.set_ylabel("Wavelet Period [Years]")
ax2.set_title("Morlet Wavelet Coefficients, Avg Max Temp Data")
save_string = 'avg_precip_data_with_wavelet_coefficients_all_stations'
#plt.savefig(save_string)
plt.show()

In [47]:
# FOR THESIS, PLOTTING SMOOTHED DATA WITH TRANSFORMED DATA
coi_data_left = coi_left(x_list[0], nei_years)
coi_data_right = coi_right(x_list[-1], nei_years)

smoothed_data = gaussian_smoother(x_list, data, kernel_std)

#size is usually 24,13
fig = plt.figure(figsize=(6,8))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(smoothed_data[0], smoothed_data[1])
ax1.set_xlim(1900, 2016)
ax1.set_xticks(np.arange(1900, 2016, 20))
ax1.set_ylabel("Z-score")
title_string_raw = "Smooth(5yr) NEI, All StateIDs"
ax1.set_title(title_string_raw)

ax2 = plt.subplot(2, 1, 2)
ax2.contourf(x_list, nei_years, transform_power)
ax2.add_patch(Polygon(coi_data_left, closed=True,fill=False,hatch='\\'))
ax2.add_patch(Polygon(coi_data_right, closed=True,fill=False,hatch='/'))
ax2.set_xlabel("Year")
ax2.set_ylabel("Wavelet Period [Years]")
ax2.set_title("Wavelet Coefficients, NEI")
save_string = 'avg_nei_data_with_wavelet_coefficients_all_stations_thesis'
plt.savefig(save_string)
plt.show()

### Applying Harr Wavelet to Various Data Sets

In [14]:
# running Harr wavelet on random data
start_time = time.perf_counter()

x_list = np.linspace(0, 116, 460)
r = 0.35
std_dev = 1
y_list_noise = red_noise_generator(r, std_dev, 1000)
y_list_values = line_generator(0, 0.00, x_list)
noisy_data = noisy_line_generator(x_list, y_list_values, y_list_noise)

#station = select_id(62658, StateID_objects)
#min_winter_temps_list = dict_to_list(station.max_temp_winter())
#N = len(min_winter_temps_list)
#x_list = np.linspace(1900, 2014+0.75, N, endpoint=True)
scales = harr_scale_range((1.0/6.0), 10)
transform_coefficients = continuous_transform_harr(noisy_data[1], scales)

end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

Z = transform_coefficients

# usually 24,13
plt.figure(figsize=(6,8))
plt.subplot(2, 1, 1)
plt.plot(x_list, noisy_data[1])
#plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Red Noise w/ r =" + str(r) + " and std_dev = " + str(std_dev)
plt.title(title_string_raw)

plt.subplot(2, 1, 2)
plt.contourf(x_list, scales, Z)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Harr Wavelet Coefficients, Raw Data")
#cbar = plt.colorbar()
save_string = 'harr_wavelet_noise_thesis'
plt.savefig(save_string)

plt.show()

Time Elapsed: 0.3879398009519264


In [29]:
# Harr wavelet on linear baseline data
start_time = time.perf_counter()

x_list = np.linspace(0, 116, 460)
r = 0.35
std_dev = 1
y_list_noise = red_noise_generator(r, std_dev, 1000)
y_list_values = line_generator(-1, 2.0/116.0, x_list)
noisy_data = noisy_line_generator(x_list, y_list_values, y_list_noise)

scales = harr_scale_range((1.0/6.0), 10)
transform_coefficients = continuous_transform_harr(noisy_data[1], scales)

end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

Z = transform_coefficients

plt.figure(figsize=(24,13))
plt.subplot(2, 1, 1)
plt.plot(x_list, noisy_data[1])
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Red Noise w/ r =" + str(r) + " and std_dev = " + str(std_dev)
plt.title(title_string_raw)

plt.subplot(2, 1, 2)
plt.contourf(x_list, scales, Z)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Harr Wavelet Coefficients, Raw Data")
cbar = plt.colorbar()

plt.show()

Time Elapsed: 0.39255864825645403


In [51]:
# Harr wavelet on line w/ slope 5 globally
start_time = time.perf_counter()

x_list = np.linspace(0, 115.75, 460)
slope = 5.0
y_list_values = line_generator(-2, slope, x_list)

#station = select_id(62658, StateID_objects)
#min_winter_temps_list = dict_to_list(station.max_temp_winter())
#N = len(min_winter_temps_list)
#x_list = np.linspace(1900, 2014+0.75, N, endpoint=True)
scales = harr_scale_range((1.0/6.0), 10)
transform_coefficients = continuous_transform_harr(y_list_values, scales)

end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

Z = transform_coefficients

plt.figure(figsize=(24,13))
plt.subplot(2, 1, 1)
plt.plot(x_list, y_list_values)
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Line w/ slope: " + str(slope)
plt.title(title_string_raw)

plt.subplot(2, 1, 2)
plt.contourf(x_list, scales, Z)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Harr Wavelet Coefficients, Raw Data")
cbar = plt.colorbar()

plt.show()

Time Elapsed: 0.40622502551195794


In [58]:
# Harr wavelet on line w/ slope 5 for first half, slope -5 for second half
start_time = time.perf_counter()

x_list = np.linspace(0, 115.75, 460)
slope1 = 5.0
slope2 = -5.0
y_list_values_1half = line_generator(0, slope1, x_list[:230])
y_list_values_2half = line_generator(290, slope2, x_list[230:])
y_list_values = y_list_values_1half + y_list_values_2half
print(len(y_list_values))
scales = harr_scale_range((1.0/6.0), 10)
transform_coefficients = continuous_transform_harr(y_list_values, scales)

end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

Z = transform_coefficients

plt.figure(figsize=(24,13))
plt.subplot(2, 1, 1)
plt.plot(x_list, y_list_values)
plt.xlabel("Year")
plt.ylabel("Z-score")
title_string_raw = "Line w/ slope: " + str(slope)
plt.title(title_string_raw)

plt.subplot(2, 1, 2)
plt.contourf(x_list, scales, Z)
plt.xlabel("Year")
plt.ylabel("Scale")
plt.title("Harr Wavelet Coefficients, Raw Data")
cbar = plt.colorbar()

plt.show()

460
Time Elapsed: 0.4119696699576707


# Random Batch Data #

In [7]:
r = 0.7
std_dev = 1
length = 1380
batch_size = 100
wavelet_scales = scale_range(0.5, 30)
raw_data = no_baseline_red_noise_batch(r, std_dev, length, batch_size)

In [5]:
x_list = np.linspace(0, 114+11.0/12, 1380, endpoint=True)

plt.figure(figsize=(24,13))
for i in range(0, len(raw_data)):
    plt.plot(x_list, raw_data[i])
    
plt.xlabel("Time")
plt.ylabel("Noise Value")
title_string_raw = "First Red Noise Set"
plt.title(title_string_raw)

plt.show()

In [8]:
start_time = time.perf_counter()

avg_wavelet_coeff = batch_wavelet_transform(raw_data)
avg_wavelet_power = wavelet_power_spectra(avg_wavelet_coeff)

avg_z_wavelet_coeff = batch_z_score_wavelet_transform(raw_data)
avg_z_wavelet_power = wavelet_power_spectra(avg_z_wavelet_coeff)

x_list = np.linspace(0, 114+11.0/12, 1380, endpoint=True)

end_time = time.perf_counter()
print("Time Elapsed: "+ str(end_time-start_time))

plt.figure(figsize=(24,13))
plt.subplot(2,1,1)
plt.contourf(x_list, wavelet_scales, avg_z_wavelet_power)
plt.xlabel("Time")
plt.ylabel("Period [Years]")
plt.title("Averaged Morlet Wavelet Coefficients of Raw Data, r=0.7, no baseline, 100 trials")
cbar = plt.colorbar()

plt.subplot(2,1,2)
plt.contourf(x_list, wavelet_scales, avg_wavelet_power)
plt.xlabel("Time")
plt.ylabel("Period [Years]")
plt.title("Corresponding Averaged Morlet Wavelet Coefficients of z-score Data")
cbar = plt.colorbar()

plt.tight_layout()
save_string = 'nobaseline_rednoise_r07_batch100_with_zscore'
plt.savefig(save_string)

plt.show()

Time Elapsed: 21528.86536401514


# Creating Database of Random Signal Power Profiles #

In [4]:
# THIS IS TEMPLATE CODE FOR RUNNING OTHER BATCH PROCESSING SETS, DO NOT RUN THIS

# doing analysis on all files in subfolder
start_time = time.perf_counter()

wavelet_scales = list(np.arange(start=0.5, stop=21.5))+list(np.arange(start=21.5, stop=43.5, step=2.0))
filenames = os.listdir('C:\\Anaconda3\\WAVELETPROJECT\\Noisy_Sim_Data')
#making batches
#first_batch = filenames[:2]
#second_batch = filenames[2:10]
#third_batch = filenames[10:20]
#fourth_batch = filenames[20:40]
#fifth_batch = filenames[40:64]
#have completed fifth batch
#sixth_batch = filenames[64:]
batch = sixth_batch

counter = 1
send_email('Data Processing Pipeline', 'Starting batch, size: '+str(len(batch)))

for i in batch:
    counter += 1
    iter_start = time.perf_counter()
    
    # actual data analysis here
    power_matrix = sim_data_transform(i, wavelet_scales)
    df = power_profile_dataframe(power_matrix, wavelet_scales)
    save_string = i.strip('.csv')+'_POWER_PROFILE.csv'
    df.to_csv(save_string)
    
    # email here
    if counter%5==0:
        iter_end = time.perf_counter()
        time_elapsed = str(iter_end-iter_start)
        email_message = 'Finished Processing: '+save_string+' in '+time_elapsed+' seconds. Currently on '+str(counter)+'of'+str(len(batch))
        email_subject = 'Data Processing Pipeline'
        send_email(email_subject, email_message)
    
end_time = time.perf_counter()
batch_elapsed_time = str(end_time-start_time)
send_email('Data Processing Pipeline', 'Finished batch in '+batch_elapsed_time+' seconds.')

KeyboardInterrupt: 

In [18]:
# doing analysis on a sparse set of files in subfolder

# wavelet scales were modified so that each file should only take ~13 minutes to process, vs ~21 minutes
wavelet_scales = list(np.arange(start=0.5,stop=21.5,step=2.0))+\
    list(np.arange(start=21.5,stop=31.5,step=2.0))+list(np.arange(start=31.5,stop=42.5,step=5.0))
filenames = os.listdir('C:\\Anaconda3\\WAVELETPROJECT\\Noisy_Sim_Data')

r_list = [0.0+0.1*i for i in range(11)]
r_keep = [r_list[0]]+r_list[1::2]
std_dev_keep = [.25]+[0.5+0.5*i for i in range(6)]

sparse_filenames = []
for i in filenames:
    name_split = i.strip('.csv').split('_')
    if (float(name_split[5]) in r_keep) and (float(name_split[7]) in std_dev_keep):
        sparse_filenames.append(i)

first_batch = sparse_filenames[:3]
second_batch = sparse_filenames[3:7]
third_batch = sparse_filenames[7:17]
fourth_batch = sparse_filenames[17:28]
fifth_batch = sparse_filenames[28:50]
sixth_batch = sparse_filenames[50:75]
seventh_batch = sparse_filenames[75:125]
eigth_batch = sparse_filenames[125:165]
ninth_batch = sparse_filenames[165:200]
final_batch = sparse_filenames[200:]
finished = []
#finished ninth batch
batch = finished

counter = 1
send_email('Data Processing Pipeline', 'Starting batch, size: '+str(len(batch)))

start_time = time.perf_counter()
for i in batch:
    counter += 1
    iter_start = time.perf_counter()
    
    # actual data analysis here
    power_matrix = sim_data_transform(i, wavelet_scales)
    df = power_profile_dataframe(power_matrix, wavelet_scales)
    save_string = i.strip('.csv')+'_POWER_PROFILE.csv'
    df.to_csv(save_string)
    
    # email here
    if counter%10==0:
        iter_end = time.perf_counter()
        time_elapsed = str(iter_end-iter_start)
        email_message = 'Finished Processing: '+save_string+' in '+time_elapsed+' seconds. Currently on '+str(counter)+'of'+str(len(batch))
        email_subject = 'Data Processing Pipeline'
        send_email(email_subject, email_message)
    
end_time = time.perf_counter()
batch_elapsed_time = str(end_time-start_time)
send_email('Data Processing Pipeline', 'Finished batch in '+batch_elapsed_time+' seconds.')

In [ ]:
# using this to write template for multiprocessing in Plotting Scripts
# MULTIPROCESSING IS MESSY IN JUPYTER, DON'T USE
"""
import multiprocessing

wavelet_scales = list(np.arange(start=0.5,stop=21.5,step=2.0))+\
    list(np.arange(start=21.5,stop=31.5,step=2.0))+list(np.arange(start=31.5,stop=42.5,step=5.0))
filenames = os.listdir('C:\\Anaconda3\\WAVELETPROJECT\\Noisy_Sim_Data')

r_list = [0.0+0.1*i for i in range(11)]
r_keep = [r_list[0]]+r_list[1::2]
std_dev_keep = [.25]+[0.5+0.5*i for i in range(6)]

sparse_filenames = []
for i in filenames:
    name = i.strip('.csv')
    name_split = name.split('_')
    if (float(name_split[5]) in r_keep) and (float(name_split[7]) in std_dev_keep):
        sparse_filenames.append(i)

batch = sparse_filenames[125:129]
counter = 1
send_email('Data Processing Pipeline', 'Starting batch, size: '+str(len(batch)))

start_time = time.perf_counter()
iter_start = time.perf_counter()
pool = multiprocessing.Pool(processes=4)
pool.map(file_to_profile, batch)
pool.close()
pool.join()
    
end_time = time.perf_counter()
batch_elapsed_time = str(end_time-start_time)
send_email('Data Processing Pipeline', 'Finished batch in '+batch_elapsed_time+' seconds.')
"""

In [5]:
# graphing profiles
test_profiles = os.listdir('C:\\Anaconda3\\WAVELETPROJECT\\Noisy_Sim_Profiles')[::50]
print(len(test_profiles))
test_profile = load_profile(test_profiles[0])
print(test_profile)

6
[(0.5, 0.7554260406340594), (2.5, 0.7741397971677345), (4.5, 0.7529014734873142), (6.5, 0.8552365614032907), (8.5, 0.9344399451244702), (10.5, 1.0544840274177811), (12.5, 1.2258450538516357), (14.5, 1.388473245213372), (16.5, 1.4544389724280185), (18.5, 1.6041734785154413), (20.5, 1.8380572599718714), (21.5, 1.9666680222907473), (23.5, 2.2476051235333174), (25.5, 2.5601905339001743), (27.5, 2.8976472215799776), (29.5, 3.2581058915373875), (31.5, 3.603181722231946), (36.5, 4.847183375260755), (41.5, 5.586628975835314)]


In [18]:
scales, avg_pow = zip(*test_profile)

plt.figure(figsize=(10,14))
N = len(test_profiles)
for i in range(N):
    scales, avg_pow = zip(*load_profile(test_profiles[i]))
    plt.subplot(N, 1,i+1)
    plt.plot(scales, avg_pow)
    plt.xlabel("Scale")
    plt.xlim(0, 42)
    plt.ylabel("Avg Wavelet Power")
    title_string_raw = test_profiles[i]
    plt.title(title_string_raw)

plt.tight_layout()
plt.show()

In [15]:
# creating coefficients for z-scored data
data = average_StateID_series(StateID_objects)
west_data = average_StateID_series(Appalachian_StateID_objects[0])
east_data = average_StateID_series(Appalachian_StateID_objects[1])

wavelet_scales = list(np.arange(start=0.5,stop=21.5,step=2.0))+\
    list(np.arange(start=21.5,stop=31.5,step=2.0))+list(np.arange(start=31.5,stop=42.5,step=5.0))
    
wvlt_coeff_maxtemp = continuous_transform_morlet(data[0], wavelet_scales, 2*np.pi)
wvlt_coeff_mintemp = continuous_transform_morlet(data[1], wavelet_scales, 2*np.pi)
wvlt_coeff_precip = continuous_transform_morlet(data[2], wavelet_scales, 2*np.pi)
wvlt_coeff_nei = continuous_transform_morlet(data[3], wavelet_scales, 2*np.pi)

west_coeff_maxtemp = continuous_transform_morlet(west_data[0], wavelet_scales, 2*np.pi)
west_coeff_mintemp = continuous_transform_morlet(west_data[1], wavelet_scales, 2*np.pi)
west_coeff_precip = continuous_transform_morlet(west_data[2], wavelet_scales, 2*np.pi)
west_coeff_nei = continuous_transform_morlet(west_data[3], wavelet_scales, 2*np.pi)

east_coeff_maxtemp = continuous_transform_morlet(east_data[0], wavelet_scales, 2*np.pi)
east_coeff_mintemp = continuous_transform_morlet(east_data[1], wavelet_scales, 2*np.pi)
east_coeff_precip = continuous_transform_morlet(east_data[2], wavelet_scales, 2*np.pi)
east_coeff_nei = continuous_transform_morlet(east_data[3], wavelet_scales, 2*np.pi)

send_email('Processing z-scored data', 'Completed processing z-scored data')

In [17]:
# creating profiles from z-scored data above
base = ['all_', 'west_', 'east_']
types = ['maxt_', 'mint_', 'precip_', 'nei_']
end = '_POWER_PROFILE.csv'

wavelet_scales = list(np.arange(start=0.5,stop=21.5,step=2.0))+\
    list(np.arange(start=21.5,stop=31.5,step=2.0))+list(np.arange(start=31.5,stop=42.5,step=5.0))
wvlt_matrices = [[wvlt_coeff_maxtemp, wvlt_coeff_mintemp, wvlt_coeff_precip, wvlt_coeff_nei], \
                [west_coeff_maxtemp, west_coeff_mintemp, west_coeff_precip, west_coeff_nei], \
                [east_coeff_maxtemp, east_coeff_mintemp, east_coeff_precip, east_coeff_nei]]

for i in range(3):
    for j in range(4):
        df = power_profile_dataframe(wvlt_matrices[i][j], wavelet_scales)
        save_string = base[i]+types[j]+end
        df.to_csv(save_string)

MWWavelets.ipynb:85: ComplexWarning: Casting complex values to real discards the imaginary part
  "Year: 1901   Value: -9999.0\n",


# Changelog
3/13/17 Split from NE Climate Database to condense methods w/o plotting scripts
- Applied Harr wavelet on line w/ slope 5 for 1st half, slope -5 for second half

3/29/17 
- Running first set of random batch data

3/31/17
- Ran first full set of 100 batch random data, with no baseline and r = 0.7

4/18/17
- Performed wavelet transforms of sorted Appalachian mountain data, plotted coefficients

8/26/17
- moved bata data analysis processing to this file, performing power profile analysis here

9/1/17
- plotting profiles